In [1]:
import pandas as pd

In [49]:
db_train = pd.read_csv('../data_bike/train.csv')
db_test = pd.read_csv('../data_bike/test.csv')
db_test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [50]:
#Видно, что в train лишние колонки, которые нужно будет убрать
print(f'shapes: train {db_train.shape}, test {db_test.shape}')

shapes: train (10886, 12), test (6493, 9)


In [51]:
db_train.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')

In [52]:
db_test.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed'],
      dtype='object')

In [54]:

db_train.drop(['casual', 'registered'], axis=1, inplace=True)
db_train.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'count'],
      dtype='object')

In [7]:
db_train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [14]:
#все данные на месте
print(f'train nulls {db_train.isnull().sum()}')
print(f'test nulls {db_test.isnull().sum()}')

train nulls datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64
test nulls datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
dtype: int64


In [57]:
#возьмем фичи из datetime
db_train['datetime_c'] =  pd.to_datetime(db_train['datetime'], format='%Y-%m-%d %H:%M:%S')
db_test['datetime_c'] =  pd.to_datetime(db_test['datetime'], format='%Y-%m-%d %H:%M:%S')
db_train.drop(['datetime'], axis=1, inplace=True)
db_test.drop(['datetime'], axis=1, inplace=True)

In [58]:

def datetime_2_features(df):
  
  df['year'] = df['datetime_c'].dt.year
  df['month'] = df['datetime_c'].dt.month
  df['day'] = df['datetime_c'].dt.day
  df['hour'] = df['datetime_c'].dt.hour
  df['min'] = df['datetime_c'].dt.minute
  df['sec'] = df['datetime_c'].dt.second
  df.drop(['datetime_c'], axis=1, inplace=True)

datetime_2_features(db_train)
datetime_2_features(db_test)
db_train.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,month,day,hour,min,sec
0,1,0,0,1,9.84,14.395,81,0.0,16,2011,1,1,0,0,0
1,1,0,0,1,9.02,13.635,80,0.0,40,2011,1,1,1,0,0
2,1,0,0,1,9.02,13.635,80,0.0,32,2011,1,1,2,0,0
3,1,0,0,1,9.84,14.395,75,0.0,13,2011,1,1,3,0,0
4,1,0,0,1,9.84,14.395,75,0.0,1,2011,1,1,4,0,0


In [62]:
#Данных очень мало, поэтому обучимся на всех

X=db_train.iloc[:,db_train.columns!='count'].values
Y=db_train['count'].values
X_test = db_test[:].values
print(X)
print(Y)

[[ 1.  0.  0. ...  0.  0.  0.]
 [ 1.  0.  0. ...  1.  0.  0.]
 [ 1.  0.  0. ...  2.  0.  0.]
 ...
 [ 4.  0.  1. ... 21.  0.  0.]
 [ 4.  0.  1. ... 22.  0.  0.]
 [ 4.  0.  1. ... 23.  0.  0.]]
[ 16  40  32 ... 168 129  88]


In [61]:
#Данные табличные, xgboost подойдет лучше большинства моделей

import xgboost as xg
xgr=xg.XGBRegressor(max_depth=8,min_child_weight=6,gamma=0.4,colsample_bytree=0.6,subsample=0.6)
xgr.fit(X,Y)

[12:59:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.6, gamma=0.4, max_depth=8, min_child_weight=6,
             subsample=0.6)

In [65]:
y_output=xgr.predict(X_test)
y_output

array([ 45.454567,  34.57296 ,  34.574806, ..., 115.99082 ,  88.489136,
        49.15723 ], dtype=float32)

In [66]:
#выведем ответ в файл

answer = pd.DataFrame({'count':(y_output)})
answer.to_csv('sub1.csv')

In [64]:
X.shape

(10886, 14)

In [67]:
#python code
import pandas as pd
db_train = pd.read_csv('../data_bike/train.csv')
db_test = pd.read_csv('../data_bike/test.csv')
db_test.head()
db_train.drop(['casual', 'registered'], axis=1, inplace=True)
db_train.columns
db_train['datetime_c'] =  pd.to_datetime(db_train['datetime'], format='%Y-%m-%d %H:%M:%S')
db_test['datetime_c'] =  pd.to_datetime(db_test['datetime'], format='%Y-%m-%d %H:%M:%S')
db_train.drop(['datetime'], axis=1, inplace=True)
db_test.drop(['datetime'], axis=1, inplace=True)

def datetime_2_features(df):
  
  df['year'] = df['datetime_c'].dt.year
  df['month'] = df['datetime_c'].dt.month
  df['day'] = df['datetime_c'].dt.day
  df['hour'] = df['datetime_c'].dt.hour
  df['min'] = df['datetime_c'].dt.minute
  df['sec'] = df['datetime_c'].dt.second
  df.drop(['datetime_c'], axis=1, inplace=True)

datetime_2_features(db_train)
datetime_2_features(db_test)
db_train.head()
X=db_train.iloc[:,db_train.columns!='count'].values
Y=db_train['count'].values
X_test = db_test[:].values

import xgboost as xg
xgr=xg.XGBRegressor(max_depth=8,min_child_weight=6,gamma=0.4,colsample_bytree=0.6,subsample=0.6)
xgr.fit(X,Y)

y_output=xgr.predict(X_test)
y_output

answer = pd.DataFrame({'count':(y_output)})
answer.to_csv('sub2.csv')

[13:09:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
